TRANSFORMER FILE

The dpft_model.py file is an implementation of the transformer paper's blind source separation architecture.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math

FEATURE_DIM_N = 128
N_HEADS = 2 
KERNEL_SIZE = (4, 4)
STRIDES = (2, 2)



# N_FFT = 128  
N_FFT=512
# HOP_LENGTH = 128
HOP_LENGTH = 256
# ORIGINAL_LENGTH = 16384
ORIGINAL_LENGTH = 65280


DROPOUT_RATE = 0.1
I_TRANSFORMER_LAYERS = 2
J_DPTF_STACKS = 2

def compute_stft_safe(x, n_fft=N_FFT, hop_length=HOP_LENGTH, win_length=None):
    if win_length is None: win_length = n_fft
    window = torch.hann_window(win_length, device=x.device)
    
    if x.shape[-1] < n_fft:
        pad_amount = n_fft - x.shape[-1]
        x = F.pad(x, (0, pad_amount))  
    
    Xm = torch.stft(
        x, n_fft=n_fft, hop_length=hop_length, win_length=win_length,
        window=window, return_complex=True
    )
    return Xm



def preprocess_signal(xm):
    """
    xm: [B, C, T]  (batch, channels, time)
    Returns:
        Xm2_4d: [B, 4, F_padded, T_new] - LogAmp/Phase stack
        Xm_original_batched: [B, F, T_new] - Complex STFT of the *first* channel
    """
    B, C, T_old = xm.shape
    Xm2_4d_list = []
    Xm_original_batched_list = []



    for b in range(B):
        sig_mix = xm[b, 0] 
        Xm_mix = compute_stft_safe(sig_mix) 
        Xm_original_batched_list.append(Xm_mix)
        
        F, T_new = Xm_mix.shape
        F_padded = F + 1
        
        channel_features = []
        for c in range(C):
            sig = xm[b, c] 
            Xm_c = compute_stft_safe(sig) 
            Xm_c = torch.cat([Xm_c, torch.zeros(1, T_new, device=Xm_c.device)], dim=0)
            Xm_abs = torch.abs(Xm_c)
            Xm_logamp = torch.log10(torch.clamp(Xm_abs, min=1e-10))
            Xm_phase = torch.angle(Xm_c)
            channel_features.append(Xm_logamp)
            channel_features.append(Xm_phase)  
        Xm2_4d_list.append(torch.stack(channel_features, dim=0))
    Xm2_4d = torch.stack(Xm2_4d_list, dim=0)
    Xm_original_batched = torch.stack(Xm_original_batched_list, dim=0)
    return Xm2_4d, Xm_original_batched 



def post_processing_block(Xs_4d,Xm_original_batched):
    Xs_logamp = Xs_4d[:, :2] 
    Xs_phase = Xs_4d[:, 2:]  
    Xs_linamp = torch.pow(10, Xs_logamp)
    M_real = Xs_linamp * torch.cos(Xs_phase)
    M_imag = Xs_linamp * torch.sin(Xs_phase)
    Xo_4d_complex = torch.complex(M_real, M_imag)
    Xo_4d_complex = Xo_4d_complex[:, :, :-1, :]
    Xo1 = Xo_4d_complex[:, 0] 
    Xo2 = Xo_4d_complex[:, 1] 
    Xo1_separated = Xo1 * Xm_original_batched
    Xo2_separated = Xo2 * Xm_original_batched
    
    return Xo1_separated, Xo2_separated

def compute_istft(X, length=ORIGINAL_LENGTH):
    window = torch.hann_window(N_FFT, device=X.device)
    xm_out = torch.istft(
        X, n_fft=N_FFT, hop_length=HOP_LENGTH, win_length=N_FFT,
        window=window, return_complex=False, length=length
    )
    return xm_out.squeeze(0)



class PermuteLayerNorm(nn.Module):
    """Applies LayerNorm over the feature dimension N after permuting [B, C, F, T] -> [B, F, T, C]"""
    def __init__(self, normalized_shape):
        super().__init__()
        self.norm = nn.LayerNorm(normalized_shape)
    def forward(self, x):
        x = x.permute(0, 2, 3, 1)
        x = self.norm(x)
        x = x.permute(0, 3, 1, 2) 
        return x

class SeparableConv2d(nn.Module):
    """
    Simulates SeparableConv2D used in the paper (II.C)
    The paper cites Keras/TensorFlow, which has a native separable layer.
    In PyTorch, this is implemented as Depthwise followed by Pointwise convolution.
    """
    def __init__(self, in_channels, out_channels, kernel_size, stride=1, padding=0):
        super().__init__()
        self.depthwise = nn.Conv2d(
            in_channels, in_channels, kernel_size=kernel_size, stride=stride, padding=padding, groups=in_channels, bias=False
        )
        self.pointwise = nn.Conv2d(in_channels, out_channels, kernel_size=1, bias=True)

    def forward(self, x):
        return self.pointwise(self.depthwise(x))


class FeatureExtractor(nn.Module):
    def __init__(self, N=FEATURE_DIM_N, K=KERNEL_SIZE, S=STRIDES):
        super().__init__()
      

        self.separable_conv1 = SeparableConv2d(2, N, kernel_size=K, stride=1, padding=(K[0]//2, K[1]//2))

        self.norm1 = PermuteLayerNorm(N)
      
        self.separable_conv_down = SeparableConv2d(N, N, kernel_size=K, stride=S)
        self.norm_down = PermuteLayerNorm(N)
      
        self.conv_final = nn.Conv2d(N, N, kernel_size=1)
        self.norm_final = PermuteLayerNorm(N)

    def forward(self, x):

        x = self.norm1(self.separable_conv1(x))

        x = self.norm_down(self.separable_conv_down(x))
        x = F.relu(x)
        
        Xe = self.norm_final(self.conv_final(x))
        return Xe

class PositionalEncoding(nn.Module):
    """Sinusoidal Positional Encoding as required by the paper (II.D, Eq. 8)"""
    def __init__(self, dim, max_len=5000):
        super().__init__()
        pe = torch.zeros(max_len, dim)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, dim, 2).float() * (-math.log(10000.0) / dim))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, Z):
        seq_len = Z.size(1)
        E = self.pe[:seq_len, :Z.size(2)]
        return Z + E.unsqueeze(0) 


class TransformerEncoder(nn.Module):
    """
    Implements I stacks of vanilla Transformer Encoder layers (Fig. 2c).
    Applies g(.) I times.
    """
    def __init__(self, dim, num_heads, dropout=DROPOUT_RATE, num_layers=I_TRANSFORMER_LAYERS):
        super().__init__()
        self.pos_encoder = PositionalEncoding(dim)

        self.layers = nn.ModuleList([self._make_g_block(dim, num_heads, dropout) for _ in range(num_layers)])

    def _make_g_block(self, dim, num_heads, dropout):
        """Implements the g(.) block (Eq. 9, 10)"""
        return nn.ModuleList([
            nn.MultiheadAttention(dim, num_heads, dropout=dropout, batch_first=True),
            nn.LayerNorm(dim), 
            nn.Linear(dim, dim * 4),
            nn.Linear(dim * 4, dim),
            nn.LayerNorm(dim),
            nn.Dropout(dropout),
            nn.Dropout(dropout)
        ])
    
    def forward(self, Z):
    
        Z_start = Z
        Z1 = self.pos_encoder(Z) 
        Z_out = Z1
        for mha, norm1, ffn1, ffn2, norm2, drop1, drop2 in self.layers:
            residual1 = Z_out
            attn_out, _ = mha(Z_out, Z_out, Z_out)
            Z_out = norm1(residual1 + drop1(attn_out)) 
            
            residual2 = Z_out
            ffw_out = ffn2(F.relu(ffn1(Z_out)))
            Z_out = norm2(residual2 + drop2(ffw_out))
    
        Z4 = Z_out + Z_start
        return Z4


class DPTFBlock(nn.Module):
    """
    Implements the h(.) module: F-TE followed by T-TE (in series) (Fig. 2b)
    Note: The paper implies shared weights across blocks (II.D)
    """
    def __init__(self, N=FEATURE_DIM_N, H=N_HEADS):
        super().__init__()
        self.f_te = TransformerEncoder(N, H)
        self.t_te = TransformerEncoder(N, H)
        
    def forward(self, x):
        B, N, F_p, T_p = x.shape 
        
        x_freq = x.permute(0, 3, 2, 1).reshape(B * T_p, F_p, N)
        x_freq = self.f_te(x_freq)
        x_out = x_freq.reshape(B, T_p, F_p, N).permute(0, 3, 2, 1)

        x_time = x_out.permute(0, 2, 3, 1).reshape(B * F_p, T_p, N)
        x_time = self.t_te(x_time)
        x_out = x_time.reshape(B, F_p, T_p, N).permute(0, 3, 1, 2)
        
        return x_out


class FeatureTransformer(nn.Module):
    """The Feature Transformer block (Fig. 2a)"""
    def __init__(self, N=FEATURE_DIM_N, num_stacks=J_DPTF_STACKS):
        super().__init__()
        self.transformer_blocks = nn.ModuleList([DPTFBlock(N) for _ in range(num_stacks)])
        
        self.conv_tanh = nn.Conv2d(N, N, kernel_size=1)
        self.conv_sigmoid = nn.Conv2d(N, N, kernel_size=1)
        self.conv_final = nn.Conv2d(N, N, kernel_size=1)
        self.norm_final = PermuteLayerNorm(N)

    def forward(self, x):
        Xt1 = x
        for block in self.transformer_blocks:
            Xt1 = block(Xt1)
            
        tanh_out = torch.tanh(self.conv_tanh(Xt1))
        sigmoid_out = torch.sigmoid(self.conv_sigmoid(Xt1))
        Xt2 = tanh_out * sigmoid_out
        
        Xt = self.norm_final(self.conv_final(Xt2))
        return Xt


class Separator(nn.Module):
    def __init__(self, N=FEATURE_DIM_N, K=KERNEL_SIZE, S=STRIDES):
        super().__init__()
        self.conv_up = nn.ConvTranspose2d(N, N, kernel_size=K, stride=S, output_padding=1)
        self.norm1 = PermuteLayerNorm(N)
    
        self.conv_final = nn.Conv2d(N, 4, kernel_size=1) 

    def forward(self, x, F_padded, T):
       
        
        x = self.conv_up(x)
        
        x = F.interpolate(x, size=(F_padded, T), mode='nearest')
        
        Xs1 = F.relu(self.norm1(x))
        
        Xs = self.conv_final(Xs1)
        return Xs


class DPFTSeparator(nn.Module):
    def __init__(self, num_transformer_stacks=J_DPTF_STACKS):
        super().__init__()
        self.feature_extractor = FeatureExtractor()
        self.feature_transformer = FeatureTransformer(num_stacks=num_transformer_stacks)
        self.separator = Separator()
        
    def forward(self, xm):
        Xm2_4d, Xm_original_batched = preprocess_signal(xm) 
        
        B, C, F_padded, T = Xm2_4d.shape
        Xe = self.feature_extractor(Xm2_4d)
        Xt = self.feature_transformer(Xe)
        Xs = self.separator(Xt, F_padded, T) 
        Xo1_separated, Xo2_separated = post_processing_block(Xs, Xm_original_batched)
        xo1 = compute_istft(Xo1_separated)
        xo2 = compute_istft(Xo2_separated)
        
        return xo1, xo2


This code uses the above transformer architecture for training of a model and testing.

In [ ]:
import torch
import torch.nn.functional as F
import itertools
import h5py
import numpy as np
import math
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader, Subset 
from dpft_model import DPFTSeparator 
from dpft_model import ORIGINAL_LENGTH
import dpft_model

# Model capacity is high (N=256, I=4, J=4) - ASSUMING these are set in dpft_model.py
FRAMES_PER_SAMPLE = 64 # 64 * 1024 = 65,536 samples (Close to ORIGINAL_LENGTH)
MAX_MIXTURES = 30 # Target max number of unique mixture samples
BATCH_SIZE = 1# Keep this small (4-8) due to the large sequence length
NUM_EPOCHS = 2 # Increased to allow for convergence

# --- STFT/PLOTTING PARAMETERS ---
N_FFT = 512
HOP_SIZE = 256
FS = 50e6 # Sample rate, assumed from paper's section III.B

# --- LOSS FUNCTIONS (SD-SDR and uPIT-SDSDR) ---

def sdsdr_loss(true_signal, estimated_signal):
    # SD-SDR calculation (Standard Scale-Dependent SDR)
    if true_signal.dim() == 0: true_signal = true_signal.unsqueeze(0)
    if estimated_signal.dim() == 0: estimated_signal = estimated_signal.unsqueeze(0)
    if true_signal.dim() == 1: true_signal = true_signal.unsqueeze(0)
    if estimated_signal.dim() == 1: estimated_signal = estimated_signal.unsqueeze(0)
    s = true_signal.view(-1)
    s_hat = estimated_signal.view(-1)
    
    s_norm_sq = torch.sum(s * s)
    s_s_hat_dot = torch.sum(s * s_hat)
    
    if s_norm_sq < 1e-8:
        s_target = torch.zeros_like(s)
    else:
        scale = s_s_hat_dot / s_norm_sq
        s_target = scale * s
        
    s_error = s_hat - s_target
    s_target_norm_sq = torch.sum(s_target * s_target)
    s_error_norm_sq = torch.sum(s_error * s_error)
    epsilon = 1e-8
    
    sdsdr = 10 * torch.log10(s_target_norm_sq / (s_error_norm_sq + epsilon) + epsilon)
    return sdsdr

def uPIT_SDSDR_Loss(true_signals, estimated_signals):
    """Calculates uPIT loss based on SD-SDR (Eq. 21)"""
    if true_signals.dim() == 2:
        true_signals = true_signals.unsqueeze(0)
    s_hat = torch.stack(estimated_signals, dim=1) 
    batch_size, num_channels, _ = true_signals.shape
    
    sd_sdr_matrix = torch.zeros(batch_size, num_channels, num_channels, device=true_signals.device)
    for i in range(num_channels):
        for j in range(num_channels):
            s_i = true_signals[:, i, :].reshape(batch_size, -1)
            s_hat_j = s_hat[:, j, :].reshape(batch_size, -1)
            sdsdr_batch = []
            for b in range(batch_size):
                sdsdr_val = sdsdr_loss(s_i[b].flatten(), s_hat_j[b].flatten()) 
                sdsdr_batch.append(sdsdr_val)
            sd_sdr_matrix[:, i, j] = torch.stack(sdsdr_batch)
            
    permutations = list(itertools.permutations(range(num_channels)))
    max_avg_sdsdr = []
    
    for b in range(batch_size):
        sdsdr_per_perm = []
        for perm in permutations:
            sdsr_sum = 0
            for i in range(num_channels):
                sdsr_sum += sd_sdr_matrix[b, i, perm[i]]
            avg_sdsdr = sdsr_sum / num_channels
            sdsdr_per_perm.append(avg_sdsdr)
            
        max_avg_sdsdr.append(torch.max(torch.stack(sdsdr_per_perm)))
        
    final_loss = -torch.mean(torch.stack(max_avg_sdsdr))
    return final_loss

# --- CUSTOM DATASET FOR ON-THE-FLY MIXING (MEMORY FIX) ---

class BSSGenerationDataset(Dataset):
    def __init__(self, I_all_frames, num_mixtures, target_length, frames_per_sample):
        self.I_all_frames = I_all_frames # All 1024-sample frames
        self.num_mixtures = num_mixtures
        self.target_length = target_length
        self.frames_per_sample = frames_per_sample
        self.num_total_frames = I_all_frames.shape[0]

        # Pre-generate random indices for each mixture to ensure fixed data split
        # This requires 2 * frames_per_sample frames per mixture
        self.frame_start_indices = np.random.randint(
            0, self.num_total_frames - self.frames_per_sample, size=(num_mixtures, 2)
        )
        
    def __len__(self):
        return self.num_mixtures

    def __getitem__(self, idx):
        # Retrieve pre-generated indices
        start_idx1 = self.frame_start_indices[idx, 0]
        start_idx2 = self.frame_start_indices[idx, 1]
        
        # 1. Concatenate Frames for S1 and S2
        S1_frames = self.I_all_frames[start_idx1 : start_idx1 + self.frames_per_sample]
        S2_frames = self.I_all_frames[start_idx2 : start_idx2 + self.frames_per_sample]
        
        # Flatten and truncate/pad to target_length (65280)
        S1 = S1_frames.flatten()[:self.target_length]
        S2 = S2_frames.flatten()[:self.target_length]
        
        # --- Data Preprocessing (Scaling, Noise, Normalization) ---
        
        # 3. Scale signals 
        s1_amp = np.random.uniform(0.1, 1.0)
        s2_amp = np.random.uniform(0.1, 1.0)
        S1_scaled = S1 * s1_amp
        S2_scaled = S2 * s2_amp
        
        # 4. Corrupt with Gaussian noise
        noise_level = np.random.uniform(0.001, 0.01)
        noise = np.random.normal(0, noise_level, self.target_length).astype(np.float32)
        
        # 5. Mix the signals
        X_M_i = S1_scaled + S2_scaled + noise 
        
        # 6. Normalize
        max_amp = np.max(np.abs(X_M_i))
        if max_amp > 1e-8:
            X_M_i /= max_amp
            S1_norm = S1_scaled / max_amp
            S2_norm = S2_scaled / max_amp
        else:
            S1_norm = S1_scaled
            S2_norm = S2_scaled

        # Convert to PyTorch tensors and ensure (C, L) shape
        X_M_tensor = torch.tensor(X_M_i, dtype=torch.float32).unsqueeze(0) # (1, L)
        Y_GT_tensor = torch.stack([
            torch.tensor(S1_norm, dtype=torch.float32), 
            torch.tensor(S2_norm, dtype=torch.float32)
        ], dim=0) # (2, L)
        
        return X_M_tensor, Y_GT_tensor

# --- PLOTTING UTILITIES ---

def compute_spectrogram(x, n_fft=N_FFT, hop_length=HOP_SIZE, fs=FS):
    """Computes and returns the magnitude spectrogram and frequency/time axis."""
    if x.dim() > 1: x = x.squeeze()
    
    window = torch.hann_window(n_fft, device=x.device)
    if x.shape[-1] < n_fft:
        pad_amount = n_fft - x.shape[-1]
        x = F.pad(x, (0, pad_amount)) 
        
    Xm = torch.stft(x, n_fft=n_fft, hop_length=hop_length, win_length=n_fft,
                    window=window, return_complex=True, normalized=False)
    
    S_mag = torch.abs(Xm)
    S_log = 10 * torch.log10(torch.clamp(S_mag, min=1e-10))
    
    freq_bins = S_log.shape[0]
    time_bins = S_log.shape[1]
    
    # Frequency axis (MHz) and Time axis (ms)
    freq_axis = np.linspace(0, fs / 2, freq_bins) / 1e6 
    time_axis = np.linspace(0, (time_bins - 1) * hop_length / fs, time_bins) * 1e3 
    
    return S_log.cpu().numpy(), freq_axis, time_axis

def plot_spectrograms(xm, s1, s2, xo1, xo2, sdsdr_val):
    """Plots the mixture, ground truth signals, and predicted separated signals as Log-Spectrograms."""
    
    fig, axes = plt.subplots(3, 2, figsize=(12, 12))
    plt.suptitle(f'Blind Source Separation Results\nAvg Max SD-SDR: {sdsdr_val:.4f} dB', fontsize=16)
    
    signals = {
        'Log-Spectrogram mixture': xm,
        'Log-Spectrogram signal 1 (GT)': s1,
        'Log-Spectrogram prediction 1': xo1,
        'Log-Spectrogram signal 2 (GT)': s2,
        'Log-Spectrogram prediction 2': xo2,
    }

    plot_order = [
        ('Log-Spectrogram mixture', axes[0, 0]),
        ('Log-Spectrogram signal 1 (GT)', axes[1, 0]),
        ('Log-Spectrogram prediction 1', axes[1, 1]),
        ('Log-Spectrogram signal 2 (GT)', axes[2, 0]),
        ('Log-Spectrogram prediction 2', axes[2, 1]),
    ]
    
    fig.delaxes(axes[0, 1]) 
    
    # Plot Mixture
    S_log_mix, freq_axis, time_axis = compute_spectrogram(signals['Log-Spectrogram mixture'])
    S_log_mix = S_log_mix[:N_FFT//2 + 1, :]
    ax = axes[0, 0]
    im = ax.imshow(S_log_mix, aspect='auto', origin='lower',
                   extent=[time_axis.min(), time_axis.max(), freq_axis.min(), freq_axis.max()],
                   cmap='viridis')
    ax.set_title(plot_order[0][0])
    ax.set_xlabel('time (ms)')
    ax.set_ylabel('freq (MHz)')
    cbar_mix = fig.colorbar(im, ax=ax, orientation='vertical', pad=0.02)
    cbar_mix.set_label('dB')
    
    # Plot Sources/Predictions
    for i in range(1, len(plot_order)):
        title, ax = plot_order[i]
        signal = signals[title]
        S_log, freq_axis, time_axis = compute_spectrogram(signal)
        S_log = S_log[:N_FFT//2 + 1, :]

        im = ax.imshow(S_log, aspect='auto', origin='lower',
                       extent=[time_axis.min(), time_axis.max(), freq_axis.min(), freq_axis.max()],
                       cmap='viridis')
        ax.set_title(title)
        ax.set_xlabel('time (ms)')
        ax.set_ylabel('freq (MHz)')
        
        cbar = fig.colorbar(im, ax=ax, orientation='vertical', pad=0.02)
        cbar.set_label('dB')
        
    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plt.savefig("separation_spectrograms.png", dpi=300)
    plt.show()

# --- MAIN EXECUTION BLOCK ---

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", DEVICE)

h5_path = r"/home/bss/data/GOLD_XYZ_OSC.0001_1024.hdf5"

try:
    with h5py.File(h5_path, "r") as f:
        print("Keys in HDF5 file:", list(f.keys()))
        X_all_np = f['X'][:]
    I_all_frames = X_all_np[:, :, 0].astype(np.float32) 
    num_total_frames = I_all_frames.shape[0]

except FileNotFoundError:
    print(f"Error: HDF5 file not found at {h5_path}. Cannot proceed without real data.")
    exit()

# Filter is removed (using all frames) to ensure maximum data volume
I_high_snr = I_all_frames 

max_feasible_mixtures = num_total_frames // (2 * FRAMES_PER_SAMPLE)
NUM_MIXTURES = min(MAX_MIXTURES, max_feasible_mixtures)

print(f"Total frames: {num_total_frames}. High SNR (0dB+) frames selected: {num_total_frames}")
print(f"Number of mixtures to generate: {NUM_MIXTURES}")
print(f"Generating {NUM_MIXTURES} synthetic BSS samples by concatenating {FRAMES_PER_SAMPLE} frames.")

# --- DATASET AND LOADER SETUP ---

# Instantiate the full dataset
full_dataset = BSSGenerationDataset(
    I_all_frames=I_high_snr,
    num_mixtures=NUM_MIXTURES,
    target_length=ORIGINAL_LENGTH,
    frames_per_sample=FRAMES_PER_SAMPLE
)

all_indices = np.arange(NUM_MIXTURES)
np.random.shuffle(all_indices)

# Define splits
train_split = int(0.8 * NUM_MIXTURES)
val_split = int(0.9 * NUM_MIXTURES)

train_indices = all_indices[:train_split]
val_indices = all_indices[train_split:val_split]
test_indices = all_indices[val_split:]

print(f"Dataset Split: Train={len(train_indices)}, Val={len(val_indices)}, Test={len(test_indices)}")

# Use Subset for splitting
train_dataset = Subset(full_dataset, train_indices)
val_dataset = Subset(full_dataset, val_indices)
test_dataset = Subset(full_dataset, test_indices)

# DataLoaders with multiprocessing (num_workers) for speed
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=1, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=1, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=1, pin_memory=True)

# --- MODEL AND TRAINING SETUP ---

def compute_stft_custom(xm, n_fft=N_FFT, hop_length=HOP_SIZE, win_length=None):
    if win_length is None: win_length = n_fft
    window = torch.hann_window(win_length, device=xm.device)
    if xm.dim() > 2: xm = xm.squeeze(1) 
    
    if xm.shape[-1] < n_fft:
        pad_amount = n_fft - xm.shape[-1]
        xm = F.pad(xm, (0, pad_amount)) 
        
    Xm = torch.stft(xm, n_fft=n_fft, hop_length=hop_length, win_length=win_length,
                    window=window, return_complex=True, normalized=False)
    return Xm

dpft_model.compute_stft = compute_stft_custom

model = DPFTSeparator()
model.to(DEVICE)
criterion = uPIT_SDSDR_Loss
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
num_epochs = NUM_EPOCHS
train_losses_history = [] 
val_losses_history = []
best_val_loss = float('inf')

# --- TRAINING LOOP ---
print("\nStarting Training with uPIT-SDSDR Loss...")
for epoch in range(num_epochs):
    model.train() 
    running_loss = 0.0
    
    # Decaying learning rate schedule (Eq. 22)
    current_lr = 1e-4 * (0.90 ** epoch)
    for param_group in optimizer.param_groups:
        param_group['lr'] = current_lr
        
    for batch_idx, (xm_batch, ym_batch) in enumerate(train_loader):
        xm_batch = xm_batch.to(DEVICE) 
        ym_batch = ym_batch.to(DEVICE) 
        
        optimizer.zero_grad()
        
        xo1, xo2 = model(xm_batch) 
        
        xo1_safe = xo1.view(xm_batch.shape[0], -1) 
        xo2_safe = xo2.view(xm_batch.shape[0], -1)
        
        loss = criterion(ym_batch, [xo1_safe, xo2_safe])
        
        # 🚨 Memory Optimization: Use torch.autocast/scaler if using mixed precision (optional but helpful)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        
    avg_train_loss = running_loss / len(train_loader)
    train_losses_history.append(avg_train_loss)
    
    # Validation
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for xm_batch, ym_batch in val_loader:
            xm_batch = xm_batch.to(DEVICE)
            ym_batch = ym_batch.to(DEVICE)
            
            xo1, xo2 = model(xm_batch)
            xo1_safe = xo1.view(xm_batch.shape[0], -1)
            xo2_safe = xo2.view(xm_batch.shape[0], -1)
            
            loss = criterion(ym_batch, [xo1_safe, xo2_safe])
            val_loss += loss.item()
            
    avg_val_loss = val_loss / len(val_loader)
    val_losses_history.append(avg_val_loss)
    
    print(f"Epoch [{epoch+1}/{num_epochs}], LR: {current_lr:.2e}, Train Loss: {avg_train_loss:.4f} dB, Val Loss: {avg_val_loss:.4f} dB")
    
    # Save best model
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        torch.save(model.state_dict(), 'best_dpft_model.pth')
        print("Model saved!")

print("Training complete.")

# --- TESTING AND PLOTTING ---
print("\nStarting Model Testing...")
try:
    # 🚨 Memory Safety: Ensure the model loads to the correct device
    model.load_state_dict(torch.load('best_dpft_model.pth', map_location=DEVICE)) 
except FileNotFoundError:
    print("Warning: Best model weights not found. Using final epoch weights.")
model.eval()
test_loss = 0.0
total_sdsdr = 0.0
sample_for_plotting = None
sdsdr_for_plotting = 0.0

with torch.no_grad():
    for xm_batch, ym_batch in test_loader:
        xm_batch = xm_batch.to(DEVICE)
        ym_batch = ym_batch.to(DEVICE)
        
        xo1, xo2 = model(xm_batch)
        
        xo1_safe = xo1.view(xm_batch.shape[0], -1)
        xo2_safe = xo2.view(xm_batch.shape[0], -1)
        
        loss = criterion(ym_batch, [xo1_safe, xo2_safe])
        test_loss += loss.item()
        total_sdsdr += -loss.item() * xm_batch.shape[0] 
        
        # Capture the first sample for plotting
        if sample_for_plotting is None:
            sample_for_plotting = (
                xm_batch[0, 0].cpu(), 
                ym_batch[0, 0].cpu(), 
                ym_batch[0, 1].cpu(), 
                xo1_safe[0].cpu(), 
                xo2_safe[0].cpu()
            )
            single_sample_loss = -criterion(ym_batch[0].unsqueeze(0), [xo1_safe[0].unsqueeze(0), xo2_safe[0].unsqueeze(0)]).item()
            sdsdr_for_plotting = single_sample_loss

avg_test_loss = test_loss / len(test_loader)
avg_test_sdsdr = total_sdsdr / len(test_indices)
print(f"\n--- Test Results ---")
print(f"Test Loss (Negative Avg SD-SDR): {avg_test_loss:.4f} dB")
print(f"Average Max SD-SDR on Test Set: {avg_test_sdsdr:.4f} dB")

# --- Plotting ---
plt.figure(figsize=(10, 6))
epochs_range = range(1, num_epochs + 1)
plt.plot(epochs_range, train_losses_history, label='Training Loss (-Avg SD-SDR)', color='blue')
plt.plot(epochs_range, val_losses_history, label='Validation Loss (-Avg SD-SDR)', color='red')
plt.title('Learning Curve (Negative Average SD-SDR) [cite: 256]')
plt.xlabel('Epoch')
plt.ylabel('Loss (dB)')
plt.legend()
plt.grid(True)
plt.savefig("training_curve.png", dpi=300, bbox_inches='tight')
plt.show()

if sample_for_plotting is not None:
    xm, s1, s2, xo1, xo2 = sample_for_plotting
    plot_spectrograms(xm, s1, s2, xo1, xo2, sdsdr_for_plotting)

training code

In [ ]:
import torch
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
import h5py
from pathlib import Path
from torch.utils.data import Dataset, DataLoader

# Import your model
from dpft_model import DPFTSeparator, ORIGINAL_LENGTH
import dpft_model

# --- CONFIG ---
H5_PATH = "/home/bss/data/GOLD_XYZ_OSC.0001_1024.hdf5"
SPLIT_DIR = Path("/home/bss/dev/")
MODEL_PATH = "best_dpft_model.pth"
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
FRAMES_PER_SAMPLE = 64
N_FFT = 512
HOP_SIZE = 256

# --- 1. RE-DEFINE DATASET (Must match training) ---
class BSSFromIndicesDataset(Dataset):
    def __init__(self, h5_data, valid_indices, num_mixtures, target_length, frames_per_sample):
        self.data = h5_data
        self.valid_indices = valid_indices
        self.num_mixtures = num_mixtures
        self.target_length = target_length
        self.frames_per_sample = frames_per_sample
        
    def __len__(self):
        return self.num_mixtures

    def __getitem__(self, idx):
        rng = np.random.default_rng(seed=idx + len(self.valid_indices))
        idx_s1 = rng.choice(self.valid_indices, size=self.frames_per_sample, replace=True)
        idx_s2 = rng.choice(self.valid_indices, size=self.frames_per_sample, replace=True)
        
        frames_s1 = self.data[np.sort(idx_s1)][:, :, 0] 
        frames_s2 = self.data[np.sort(idx_s2)][:, :, 0]
        
        S1 = frames_s1.flatten()[:self.target_length]
        S2 = frames_s2.flatten()[:self.target_length]
        
        s1_amp = rng.uniform(0.1, 1.0); S1_scaled = S1 * s1_amp
        s2_amp = rng.uniform(0.1, 1.0); S2_scaled = S2 * s2_amp
        noise = rng.normal(0, rng.uniform(0.001, 0.01), self.target_length).astype(np.float32)
        X_M_i = S1_scaled + S2_scaled + noise
        
        max_amp = np.max(np.abs(X_M_i))
        if max_amp > 1e-8:
            X_M_i /= max_amp; S1_norm = S1_scaled / max_amp; S2_norm = S2_scaled / max_amp
        else:
            S1_norm, S2_norm = S1_scaled, S2_scaled

        return (torch.tensor(X_M_i, dtype=torch.float32).unsqueeze(0), 
                torch.stack([torch.tensor(S1_norm), torch.tensor(S2_norm)]))

# --- 2. SETUP MODEL & DATA ---
# Monkey-patch STFT for model compatibility
def compute_stft_custom(xm, n_fft=N_FFT, hop_length=HOP_SIZE, win_length=None):
    if win_length is None: win_length = n_fft
    window = torch.hann_window(win_length, device=xm.device)
    if xm.dim() > 2: xm = xm.squeeze(1) 
    if xm.shape[-1] < n_fft: xm = F.pad(xm, (0, n_fft - xm.shape[-1])) 
    return torch.stft(xm, n_fft, hop_length, win_length, window, return_complex=True)

dpft_model.compute_stft = compute_stft_custom

print("Loading Data & Indices...")
f = h5py.File(H5_PATH, "r"); X_all = f['X'][:]; f.close()
idx_test = np.load(SPLIT_DIR / "test_indices.npy")
test_ds = BSSFromIndicesDataset(X_all, idx_test, 10, ORIGINAL_LENGTH, FRAMES_PER_SAMPLE) # Load 10 samples
test_loader = DataLoader(test_ds, batch_size=1, shuffle=False)

print("Loading Model...")
model = DPFTSeparator().to(DEVICE)
try:
    model.load_state_dict(torch.load(MODEL_PATH, map_location=DEVICE))
    print("Weights Loaded ✔")
except:
    print("⚠️ Could not load weights. Using random init.")

model.eval()

# --- 3. GENERATE PLOTS ---
print("Generating visualizations...")
# Get one batch (1 sample)
xm, ym = next(iter(test_loader))
xm, ym = xm.to(DEVICE), ym.to(DEVICE)

with torch.no_grad():
    xo1, xo2 = model(xm)
    # Handle dimensions
    if xo1.dim()==1: xo1 = xo1.unsqueeze(0)
    if xo2.dim()==1: xo2 = xo2.unsqueeze(0)

# Convert to Numpy for plotting
mix = xm[0].cpu().numpy().flatten()
s1_gt = ym[0,0].cpu().numpy().flatten()
s2_gt = ym[0,1].cpu().numpy().flatten()
s1_pred = xo1[0].cpu().numpy().flatten()
s2_pred = xo2[0].cpu().numpy().flatten()

# Plot 1: Waveforms
plt.figure(figsize=(12, 8))
plt.subplot(3, 1, 1); plt.plot(mix, color='grey'); plt.title("Input Mixture"); plt.grid(alpha=0.3)
plt.subplot(3, 1, 2); plt.plot(s1_gt, 'g', label='GT', alpha=0.6); plt.plot(s1_pred, 'k--', label='Pred', alpha=0.8); plt.title("Source 1"); plt.legend()
plt.subplot(3, 1, 3); plt.plot(s2_gt, 'b', label='GT', alpha=0.6); plt.plot(s2_pred, 'k--', label='Pred', alpha=0.8); plt.title("Source 2"); plt.legend()
plt.tight_layout()
plt.savefig("eval_waveforms_dpft.png")
print("Saved eval_waveforms_dpft.png")

# Plot 2: Spectrograms
def plot_spec(ax, sig, title):
    ax.specgram(sig, NFFT=256, Fs=1, noverlap=128, cmap='magma')
    ax.set_title(title)

fig, axes = plt.subplots(3, 2, figsize=(10, 10))
# Row 1: Mix
plot_spec(axes[0,0], mix, "Mixture")
axes[0,1].axis('off')
# Row 2: Source 1
plot_spec(axes[1,0], s1_gt, "GT Src 1")
plot_spec(axes[1,1], s1_pred, "Pred Src 1")
# Row 3: Source 2
plot_spec(axes[2,0], s2_gt, "GT Src 2")
plot_spec(axes[2,1], s2_pred, "Pred Src 2")
plt.tight_layout()
plt.savefig("eval_spectrograms_dpft.png")
print("Saved eval_spectrograms_dpft.png")